In [55]:
import boto3
import pandas as pd
import numpy as np

import os
import subprocess

In [28]:
# Some constants
BUCKET_NAME = 'job-site-scrapers-home'

BUCKET_NAME_RESULT = 'job-scraping-result'

In [51]:

bucket_name = BUCKET_NAME
local_directory = 'scrapers'


def download_folder(bucket_name, folder_name, local_dir):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

    for obj in response.get('Contents', []):
        file_key = obj.get('Key')
        local_path = os.path.join(local_dir, file_key)
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        s3.download_file(bucket_name, file_key, local_path)

def list_s3_top_level_folders(bucket_name):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

    folders = []
    for prefix in response.get('CommonPrefixes', []):
        folder_name = prefix.get('Prefix')
        folders.append(folder_name)

    return folders


# let clear the local folder before download
# get folder called scrapers and empty it
subprocess.run(['rm', '-rf', 'scrapers'])
subprocess.run(['mkdir', 'scrapers'])

folder_list = list_s3_top_level_folders(bucket_name)
for folder in folder_list:
    download_folder(bucket_name, folder.rstrip('/'), local_directory)


In [57]:
# upload to a bucket called job-scraping-result

def UploadJobScrapingResult(folder_name, file, file_path):
    s3 = boto3.client('s3')
    s3.upload_file(file_path, BUCKET_NAME_RESULT, folder_name + '/' + file)

In [58]:
# get the result inside each scraper folder

def GetResultandUploadS3(result_folder_path, folder_name):

    # list all the files inside result folder
    result_files = os.listdir(result_folder_path)

    csv_files = [file for file in result_files if file.endswith('.csv')]

    # add the date to folder_name
    folder_name = folder_name + '-' + str(pd.datetime.now().date())

    # upload the file to s3
    for file in csv_files:
        file_path = f'{result_folder_path}/{file}'
        UploadJobScrapingResult(folder_name=folder_name, file=file, file_path=file_path)



In [59]:
folder_path = os.path.join(os.getcwd(), 'scrapers', folder)

# folder name of folder_path
folder_name = folder_path.split('/')[-1]
print('folder_name:', folder_name)

result_folder_path = os.path.join(folder_path, 'result')

print(result_folder_path)


GetResultandUploadS3(result_folder_path, folder_name)

folder_name: linkedin-scraper
/home/thieu/DataScience2/AutoJobSubmittion/run-everyday-stuff/scrapers/linkedin-scraper/result


/tmp/ipykernel_17468/3890162792.py:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  folder_name = folder_name + '-' + str(pd.datetime.now().date())


In [56]:
# get a folder called scrapers
scrapers_folder = os.listdir('scrapers')

# loop through all folder inside that scrapers folder
for folder in scrapers_folder:
    print(folder)

    # get the folder path of the folder
    folder_path = os.path.join(os.getcwd(), 'scrapers', folder)

    # get the script path: main.py in each folder
    script_path = os.path.join(folder_path, 'main.py')

    # run the script
    
    result = subprocess.run(['python', script_path], cwd=folder_path, capture_output=True)
    print(result.stdout.decode('utf-8'))    

    # get the result folder path inside the folder_path
    result_folder_path = os.path.join(folder_path, 'result')
    GetResultandUploadS3(result_folder_path)


linkedin-scraper
Cookie loaded
start scraping that
what the hell
data%20intern
Empty DataFrame
Columns: [job_title, job_link, description, company_name, company_link, job_location, workplace_type, post_date, number_of_applicants, skill_list, title]
Index: []
https://www.linkedin.com/jobs/search/?geoId=103644278&keywords=data%20intern&location=United%20States&refresh=true&sortBy=R
Show 3 results
find job
[<selenium.webdriver.remote.webelement.WebElement (session="c6ed9544db17a864a4fc32f96b8914bc", element="C23659429FD1E002885C1A221FE32932_element_55")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ed9544db17a864a4fc32f96b8914bc", element="C23659429FD1E002885C1A221FE32932_element_56")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ed9544db17a864a4fc32f96b8914bc", element="C23659429FD1E002885C1A221FE32932_element_57")>, <selenium.webdriver.remote.webelement.WebElement (session="c6ed9544db17a864a4fc32f96b8914bc", element="C23659429FD1E002885C1A221FE32932_ele

TypeError: GetResultandUploadS3() missing 1 required positional argument: 'folder_name'